In [ ]:
!pip install julep -U --quiet

In [ ]:
import uuid
import os
import yaml
from julep import Client
from google.colab import userdata

In [ ]:
JULEP_API_KEY = userdata.get("JULEP_API_KEY")
BRAVE_KEY = userdata.get("BRAVE_API_KEY")
OPENWEATHERMAP_API_KEY = userdata.get("OPENWEATHER_API_KEY")

In [ ]:
AGENT_ID = uuid.uuid4()
TASK_ID = uuid.uuid4()

#### Creating Julep Client

In [ ]:
client = Julep(
    api_key=JULEP_API_KEY
)

#### Creating an agent

In [ ]:
agent = client.agents.create_or_update(
    agent_id = AGENT_ID,
    name = "Zesta",
    about = "A culinary travel guide that crafts delightful one-day foodie itineraries—curating breakfast, lunch, and dinner experiences based on local weather and vibes. Perfect for food lovers seeking comfort and flavor, rain or shine.",
    model = "gpt-4o",
)

#### Defining a task

In [ ]:
# Defining the foodie tour task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep One-Day Foodie Tour Planner
description: A task that creates a delightful one-day foodie tour with breakfast, lunch, and dinner recommendations based on weather conditions and local iconic dishes.

########################################################
####################### INPUT SCHEMA ##################
########################################################
input_schema:
  type: object
  properties:
    city:
      type: string
      description: The city to create a foodie tour for.
    date:
      type: string
      description: The date for the foodie tour (optional, defaults to today).

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

- name: weather
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: {OPENWEATHERMAP_API_KEY}

- name: internet_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: {BRAVE_KEY}

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Fetch today's weather for the location
- over: $ steps[0].input.location
  map:
    tool: weather
    arguments:
      location: $ _

# Step 1: Search Wikipedia for iconic local dishes of the location
- over: $ steps[0].input.location
  map:
    tool: internet_search
    arguments:
      query: $ 'iconic local dishes of ' + _

# Step 2: Zip locations, weather, and iconic_local_dishes into a list of tuples [(locationn, weather, iconic_local_dishes)]
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.locations,
          [output['result'] for output in steps[0].output],
          steps[1].output
        )
      )

# Step 3: Search for top-rated restaurants serving each of the 3 iconic dishes
- over: $ _['zipped']
  parallelism: 3
  map:
    tool: internet_search
    arguments:
      query: >-
        $ f'best restaurants in ' + {{_[0]}} + ' serving ' + {{_}}

# Step 4: Zip locations, weather, iconic_local_dishes, and restaurant_search_results into a list of tuples [(locationn, weather, iconic_local_dishes, restaurant_search_results)]
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.locations,
          [output['result'] for output in steps[0].output],
          steps[1].output,
          steps[3].output
        )
      )

# Step 4: Create the final foodie tour itinerary
- over: $ _['zipped']
  parallelism: 3
  map:
    prompt:
    - role: system
      content: >-
        $ f'''You are {{agent.name}}, a delightful food tour guide and storyteller.
        Your task is to create an enchanting one-day foodie tour for {{steps[0].input.city}}.

        You have the following information:
        - The location
        - Current weather conditions
        - 3 iconic local dishes
        - Top-rated restaurants for each dish

        Create a narrative foodie tour with:
        - Breakfast recommendation (featuring one iconic dish)
        - Lunch recommendation (featuring another iconic dish)
        - Dinner recommendation (featuring the third iconic dish)

        For each meal:
        - Consider the weather for indoor/outdoor dining suggestions
        - Include restaurant recommendations with brief descriptions
        - Add storytelling elements about the dish's cultural significance
        - Provide timing suggestions
        - Include walking/transport tips between locations

        Write in an engaging, narrative style that makes the reader excited about the food adventure.'''
    - role: user
      content: >-
        $ f'''City: {{_[0]}}
        Weather: "{{_[1]}}"

        Iconic Dishes:
        {{_[2]}}

        Restaurant Search Results:
        {{_[3]}}
        respectively for the iconic dishes

        Please create a delightful one-day foodie tour itinerary.'''
    unwrap: true

# Step 6: Format the final output
- evaluate:
    final_plan: |-
        $ '\\n---------------------------\\n'.join(activity for activity in _)
  """)

#### Creating the task object

In [ ]:
task = client.tasks.create_or_update(
    task_id = TASK_ID,
    agent_id = AGENT_ID,
    **task_def
)

#### Creating an Execution

In [ ]:
execution = client.executions.create(
    task_id = TASK_ID,
    input = {
        "locations": ["Berlin", "London", "New York", "Paris"]
    }
)
print(f"Started an execution. Execution ID: {execution.id}")

In [ ]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

if 'final_plan' in execution.output:
    print(execution.output['final_plan'])
else:
    print(execution.output)